In [1]:
import pandas as pd

In [42]:
all_data1 = pd.read_csv("data/Labor_2018_HJ1.csv", sep=";", encoding="latin1")
print(len(all_data1))
all_data2 = pd.read_csv("data/Labor_EASK_2018_2HJ.csv", sep=";", encoding="latin1")
print(len(all_data2))

1048575
1048575


In [43]:
print(all_data2["ZEITERBRACHTEL"].unique())

[nan]


In [44]:
all_data2 = all_data2[['FALLNR', 'VORGANGSNR', 'DATUMERBRACHTEL', 'CODE', 'BEZEICHNUNG',
       'BEFUNDEL', 'EINHEIT', 'NORMALWERTMIN', 'NORMALWERTMAX', 'ANMERKUNG',
       'Uhrzeit', 'Minute']]

In [45]:
print(all_data2.columns,"\n"*2, all_data1.columns)

Index(['FALLNR', 'VORGANGSNR', 'DATUMERBRACHTEL', 'CODE', 'BEZEICHNUNG',
       'BEFUNDEL', 'EINHEIT', 'NORMALWERTMIN', 'NORMALWERTMAX', 'ANMERKUNG',
       'Uhrzeit', 'Minute'],
      dtype='object') 

 Index(['FALLNR', 'VORGANGSNR', 'DATUMERBRACHTEL', 'CODE', 'BEZEICHNUNG',
       'BEFUNDEL', 'EINHEIT', 'NORMALWERTMIN', 'NORMALWERTMAX', 'ANMERKUNG',
       'Uhrzeit', 'Minute'],
      dtype='object')


In [46]:
all_data = all_data1.append(all_data2).reset_index()

In [47]:
len(all_data)

2097150

In [55]:
mibi = all_data.query("CODE==55000")

In [56]:
print(len(mibi))
mibi = mibi.append(all_data.query("CODE==10205"))
print(len(mibi))

13746
15850


In [57]:
print(len(mibi))
mibi = mibi.append(all_data.query("CODE==10200")).copy()
print(len(mibi))

15850
17997


In [58]:
mibi["ANMERKUNG"] = mibi["ANMERKUNG"].str.replace("_x000d_","")


In [59]:
mibi["ANMERKUNG"] = mibi["ANMERKUNG"].str.replace("GEWÜNSCHTE UNTERSUCHUNG:\n .*\n","")
mibi["ANMERKUNG"] = mibi["ANMERKUNG"].str.replace(" species","")
mibi["ANMERKUNG"] = mibi["ANMERKUNG"].str.replace(" sp.","")
mibi["ANMERKUNG"] = mibi["ANMERKUNG"].str.replace("\.\.\.*"," ")

In [60]:
mibi = mibi[["FALLNR","ANMERKUNG"]]

In [61]:
mibi.to_csv("data/MIBI.csv",encoding="latin1",sep=";",index=False)

In [12]:
lab = all_data.loc[~all_data.index.isin(mibi.index)].copy()

In [14]:
lab=lab[~lab["DATUMERBRACHTEL"].isin(["p"])]

In [17]:
lab["Datum"]=lab["DATUMERBRACHTEL"].astype(str)+" "+lab["Uhrzeit"].astype(str)+":"+lab["Minute"].astype(str)+":00"
lab["Referenzbereich"]=lab["NORMALWERTMIN"].astype(str)+"-"+lab["NORMALWERTMAX"]
krea_mask = lab["BEZEICHNUNG"]=="Kreatinin im Serum"
lab["Referenzbereich"][krea_mask] = (lab["NORMALWERTMIN"][krea_mask].astype(float)*88.4).astype(str)+"-"+(lab["NORMALWERTMAX"][krea_mask].astype(float)*88.4).astype(str)

C:\Users\tpatz\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [18]:
lab.to_csv("data/Labor_2018.csv",encoding="latin1",sep=";",index=False)